In [34]:
import pandas as pd
import os

In [36]:
# Set the working directory
working_directory = os.getcwd()
print(f"Current working directory: {working_directory}")

Current working directory: /Users/buboyencarnacion


In [38]:
# Load datasets
customers_df = pd.read_csv(os.path.join(working_directory, 'updated_customers_data.csv'))
products_df = pd.read_csv(os.path.join(working_directory, 'updated_products_data.csv'))
transactions_df = pd.read_csv(os.path.join(working_directory, 'updated_transactions_data.csv'))

In [40]:
### Cleaning the Datasets ###

# Products: Clean 'Product_Price' (remove "Php" and convert to numeric)
products_df['Product_Price'] = products_df['Product_Price'].replace('[\\$,]', '', regex=True).astype(float)

# Transactions: Standardize 'Transaction_Date' and handle missing values
transactions_df['Transaction_Date'] = pd.to_datetime(transactions_df['Transaction_Date'], errors='coerce')

# Fill missing numerical values in Transactions
transactions_df.fillna(transactions_df.median(numeric_only=True), inplace=True)

# Customers: Handle missing values
customers_df.fillna(customers_df.mode().iloc[0], inplace=True)

In [42]:
### Descriptive Statistics ###

# Generate descriptives for each dataset
customers_desc = customers_df.describe(include='all')
products_desc = products_df.describe(include='all')
transactions_desc = transactions_df.describe(include='all')

# Save cleaned datasets
customers_df.to_csv(os.path.join(working_directory, 'cleaned_customers_data.csv'), index=False)
products_df.to_csv(os.path.join(working_directory, 'cleaned_products_data.csv'), index=False)
transactions_df.to_csv(os.path.join(working_directory, 'cleaned_transactions_data.csv'), index=False)

# Print descriptives
print("Customers Descriptive Statistics:")
print(customers_desc)

print("\nProducts Descriptive Statistics:")
print(products_desc)

print("\nTransactions Descriptive Statistics:")
print(transactions_desc)

Customers Descriptive Statistics:
       Company_ID          Company_Name  Company_Profit  \
count         100                   100      100.000000   
unique         91                   100             NaN   
top       Unknown  Tech  Enterprises  1             NaN   
freq           10                     1             NaN   
mean          NaN                   NaN    76268.620000   
std           NaN                   NaN    25590.952639   
min           NaN                   NaN    30663.000000   
25%           NaN                   NaN    54984.250000   
50%           NaN                   NaN    75301.500000   
75%           NaN                   NaN    99467.250000   
max           NaN                   NaN   118114.000000   

                                             Address Profitability_Category  \
count                                            100                    100   
unique                                            97                      3   
top     Ayala Avenue

In [44]:
## Convert IDs to String and Ensure No Missing Values:
# Convert Product_ID to string in both datasets
products_df['Product_ID'] = products_df['Product_ID'].astype(str)
transactions_df['Product_ID'] = transactions_df['Product_ID'].astype(str)

# Convert Company_ID to string in both datasets
customers_df['Company_ID'] = customers_df['Company_ID'].astype(str)
transactions_df['Company_ID'] = transactions_df['Company_ID'].astype(str)

# Ensure no missing values in key columns
assert not transactions_df[['Product_ID', 'Company_ID']].isnull().any().any()
assert not customers_df[['Company_ID']].isnull().any().any()
assert not products_df[['Product_ID']].isnull().any().any()

In [46]:
# Merge Datasets
# Step 1: Merge Transactions with Products on 'Product_ID'
merged_df = transactions_df.merge(products_df, on='Product_ID')

# Step 2: Merge the result with Customers on 'Company_ID'
merged_df = merged_df.merge(customers_df, on='Company_ID')

# Drop unnecessary columns (optional)
merged_df.drop(columns=['Unnamed: 0'], inplace=True, errors='ignore')

# Save the merged dataset (optional)
merged_df.to_csv(os.path.join(working_directory, 'merged_dataset.csv'), index=False)

# Display the first few rows of the merged dataset
print("\nMerged DataFrame:")
print(merged_df.head())


Merged DataFrame:
   Transaction_ID Company_ID Product_ID  Quantity Transaction_Date  \
0             1.0       88.0        6.0      11.0       2024-03-26   
1             2.0       29.0       19.0      16.0              NaT   
2             4.0       85.0       12.0      12.0              NaT   
3             5.0       47.0        3.0       8.0              NaT   
4             6.0       80.0       11.0       4.0       2021-07-12   

   Product_Price_x  Total_Cost  Recency  Purchase_Frequency  Total_Spending  \
0    194379.147964   1075200.0    216.0                82.0     128878400.0   
1     97930.993380   1428000.0    730.5                69.0     121996000.0   
2    131297.783516   1008000.0    730.5                89.0     126061600.0   
3     99575.609634    705600.0    730.5                60.0      86553600.0   
4    160658.675350    627200.0   1204.0                90.0     150315200.0   

           Product_Name  Product_Price_y Price_Range           Company_Name  \
0  Rev

In [48]:
# Address Missing Data in the Merged Dataset
# Check for missing values
missing_values = merged_df.isnull().sum()
print("\nMissing Values in Merged DataFrame:")
print(missing_values[missing_values > 0])

# Numerical columns: Continuous and Count Data
# Use median for imputation
for col in merged_df.select_dtypes(include=['float64', 'int64']).columns:
    merged_df[col].fillna(merged_df[col].median(), inplace=True)

# Categorical columns: Impute with the mode
for col in merged_df.select_dtypes(include=['object']).columns:
    merged_df[col].fillna(merged_df[col].mode()[0], inplace=True)

# Date columns: Forward-fill or backward-fill for imputation
merged_df.fillna(method='ffill', inplace=True)

# Validate that missing data is resolved.
assert not merged_df.isnull().any().any()


Missing Values in Merged DataFrame:
Transaction_Date    6199
dtype: int64


/var/folders/yw/ghyqsycn5z34j1zd_gpjk9bw0000gn/T/ipykernel_12297/3355706019.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df[col].fillna(merged_df[col].median(), inplace=True)
/var/folders/yw/ghyqsycn5z34j1zd_gpjk9bw0000gn/T/ipykernel_12297/3355706019.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setti

In [50]:
## Part 2: Feature Engineering and Machine Learning Model
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

In [52]:
# Load the merged dataset
merged_df = pd.read_csv(os.path.join(working_directory, 'merged_dataset.csv'))

In [54]:
### Feature Engineering ###

# Define X (features) and y (target)
X = merged_df.drop(columns=['Product_ID'])
y = merged_df['Product_ID']

# Convert target variable to categorical (if necessary)
y = y.astype('category')

# One-Hot Encoding for Categorical Columns
X = pd.get_dummies(X, drop_first=True)

# Scale Numerical Features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Final Check: Ensure all features are processed
print("Features and Target Prepared")

Features and Target Prepared


In [56]:
### Split Data ###
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [58]:
### Model 1: Multinomial Logistic Regression ###
logistic_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
logistic_model.fit(X_train, y_train)

# Predictions
logistic_predictions = logistic_model.predict(X_test)

# Evaluation Metrics
logistic_accuracy = accuracy_score(y_test, logistic_predictions)
logistic_report = classification_report(y_test, logistic_predictions)

print("\nMultinomial Logistic Regression Results:")
print(f"Accuracy: {logistic_accuracy}")
print(logistic_report)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



Multinomial Logistic Regression Results:
Accuracy: 1.0
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00        85
         2.0       1.00      1.00      1.00        86
         3.0       1.00      1.00      1.00        84
         4.0       1.00      1.00      1.00        97
         5.0       1.00      1.00      1.00        90
         6.0       1.00      1.00      1.00        81
         7.0       1.00      1.00      1.00        78
         9.0       1.00      1.00      1.00        72
        10.0       1.00      1.00      1.00       270
        11.0       1.00      1.00      1.00        91
        12.0       1.00      1.00      1.00        67
        13.0       1.00      1.00      1.00        83
        14.0       1.00      1.00      1.00        78
        15.0       1.00      1.00      1.00        72
        17.0       1.00      1.00      1.00        92
        18.0       1.00      1.00      1.00        69
        19.0       1.00  